# Praktyka i Teoria Szeregowania Zadań
## Sprawozdanie I - Generator
### Wojcech Taisner 132330

# Teoria

## Generacja dwuetapowa
- zadania inicjalizujące - generuje zadania podstawowe, które będą wypełniać znaczącą większość czasu okreslonego
- zadania uzupełniające - generuje krótkie zadania uzupełniające, które powinny mieć zbliżone momenty rozpoczęcia do zdań inicjalizujących - będą generować one spóźnienia

In [3]:
import random

In [1]:
def generate_instance(n: int, prop = 1/2):
    total_duration = n * 2
    
    processors = [1, 1, 1, 1]
    
    regulars = int(n*prop)
    
    # generate number of processes per instance
    for _ in range(regulars - 4):
        processors[random.randrange(0, len(processors))] += 1
        
    # generuj zadania inicjalizujące
    tasks = []
    
    for i, p in enumerate(processors):
        tasks.append([])
        avg = total_duration//p
        for k in range(0, total_duration, avg):
            duration = avg + random.randint(0, 1)
            ready = max(random.randint(0,4), k-(avg//2))
            due = max(ready+duration, k + duration + random.randint(0, 4))
            
            task = (duration, ready, due )
            tasks[i].append(task)
            
    current_number_of_tasks = sum(list(map(len, tasks)))
    
    # transformuj do jednowymiarowej listy
    all_tasks = []
    for t in tasks:
        all_tasks.extend(t)
    
    # ready times dla zadań uzupełniających
    ready_times = list(map(lambda x: x[2], all_tasks))
    # wybierz tylko ready times z pewnego zakresu (żeby nie były blisko początku lub końca)
    ready_times = list(filter(lambda x: x>total_duration//6 and x<total_duration//6*5, ready_times))
    
    # generuj zadania uzupełniające
    for _ in range(n - current_number_of_tasks):
        ready = random.choice(ready_times) + random.randint(0,2)
        processing = random.randint(2,4)
        due = ready + processing + random.randint(0,2)
        processing = min(processing, due - ready)
        
        task = (processing, ready, due)
        all_tasks.append(task)
        
    # shuffle list
    random.shuffle(all_tasks)
    
    return all_tasks
        

In [4]:
for n in range(50, 501, 50):
    assert len(generate_instance(n)) == n

In [5]:
resultpath = './input/wt_test/'
# główna pętla
for n in range(50, 501, 50):
    # generuj zadania
    tasks = generate_instance(n)
    # zapis
    filepath = resultpath+'n'+str(n)+'.txt'
    with open(filepath, 'w+') as f:
        f.write(str(n)+'\n')
        for t in tasks:
            f.write(' '.join(tuple(map(str,t)))+'\n')